## Learning OpenCV 4 Computer Vision with Python 3 - Pg 93
https://github.com/PacktPublishing/Learning-OpenCV-4-Computer-Vision-with-Python-Third-Edition/blob/master/chapter04/disparity.py

In [1]:
import cv2
import numpy as np

# Setting Parameters
minDisparity = 16 # Minimum possible disparity value. Normally, it is zero but sometimes rectification algorithms can shift images, so this parameter needs to be adjusted accordingly.
numDisparities = 192-minDisparity # Maximum disparity minus minimum disparity. The value is always greater than zero. In the current implementation, this parameter must be divisible by 16.
blockSize = 5 # Matched block size. It must be an odd number >=1 . Normally, it should be somewhere in the 3..11 range
'''
uniquenessRatio:
    Margin in percentage by which the best (minimum) computed cost function value should "win" the second best value to
    consider the found match correct.
    Normally, a value within the 5-15 range is good enough.
'''
uniquenessRatio = 1 # 
speckleWindowSize = 3 # Maximum size of smooth disparity regions to consider their noise speckles and invalidate. Set it to 0 to disable speckle filtering. Otherwise, set it somewhere in the 50-200 range.
speckleRange = 3 # Maximum disparity variation within each connected component. If you do speckle filtering, set the parameter to a positive value, it will be implicitly multiplied by 16. Normally, 1 or 2 is good enough.
disp12MaxDiff = 200
'''
P1, P2
    The second parameter controlling the disparity smoothness. The larger the values are, the smoother the disparity is.
    P1 is the penalty on the disparity change by plus or minus 1 between neighbor pixels.
    P2 is the penalty on the disparity change by more than 1 between neighbor pixels.
    The algorithm requires P2 > P1 .
    See stereo_match.cpp sample where some reasonably good P1 and P2 values are shown
    (like 8*number_of_image_channels*blockSize*blockSize and 32*number_of_image_channels*blockSize*blockSize , respectively).
'''
P1 = 8 * 3 * blockSize ** 2
P2 = 32 * 3 * blockSize ** 2 # P2 must be greater than P1
# MODE (unused) - Set it to StereoSGBM::MODE_HH to run the full-scale two-pass dynamic programming algorithm. It will consume O(W*H*numDisparities) bytes, which is large for 640x480 stereo and huge for HD-size pictures. By default, it is set to false .

# Semi-Global Block Matching
stereo = cv2.StereoSGBM_create(
    minDisparity=minDisparity,
    numDisparities=numDisparities,
    blockSize=blockSize,
    uniquenessRatio=uniquenessRatio,
    speckleRange=speckleRange,
    speckleWindowSize=speckleWindowSize,
    disp12MaxDiff=disp12MaxDiff,
    P1=P1,
    P2=P2
)

imgL = cv2.imread('1lsm.png')
imgR = cv2.imread('1rsm.png')

def update(sliderValue = 0):
    '''
    This is updating the Trackbar values
    '''
    stereo.setBlockSize(
        cv2.getTrackbarPos('blockSize', 'Disparity'))
    stereo.setUniquenessRatio(
        cv2.getTrackbarPos('uniquenessRatio', 'Disparity'))
    stereo.setSpeckleWindowSize(
        cv2.getTrackbarPos('speckleWindowSize', 'Disparity'))
    stereo.setSpeckleRange(
        cv2.getTrackbarPos('speckleRange', 'Disparity'))
    stereo.setDisp12MaxDiff(
        cv2.getTrackbarPos('disp12MaxDiff', 'Disparity'))

    disparity = stereo.compute(
        imgL, imgR).astype(np.float32) / 16.0

    cv2.imshow('Left', imgL)
    cv2.imshow('Right', imgR)
    cv2.imshow('Disparity', (disparity - minDisparity) / numDisparities)

# Create Window and Trackbars
cv2.namedWindow('Disparity')
cv2.createTrackbar('blockSize', 'Disparity', blockSize, 21, update) # Keep an eye on this, must be odd number > 1
cv2.createTrackbar('uniquenessRatio', 'Disparity', uniquenessRatio, 50, update)
cv2.createTrackbar('speckleWindowSize', 'Disparity', speckleWindowSize, 200, update)
cv2.createTrackbar('speckleRange', 'Disparity', speckleRange, 50, update)
cv2.createTrackbar('disp12MaxDiff', 'Disparity', disp12MaxDiff, 250, update)

# Initialize the disparity map. Show the disparity map and images.
update()

# Wait for the user to press any key.
# Meanwhile, update() will be called anytime the user moves a slider.
cv2.waitKey()
cv2.destroyAllWindows()

https://vision.middlebury.edu/stereo/data/